<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 1 / NOTE 3*

# 📝 Assignment
---

# Ridge Regression

이번 모듈에서 배운 내용을 조금 다른 데이터를 통해 복습해 봅시다. 데이터에 대한 정보는 다음 링크를 참조해 주세요.

- [Melbourne Housing Market]('https://www.kaggle.com/anthonypino/melbourne-housing-market')

이번에도 Price 변수를 target으로 정하고 모델을 구성해 봅시다.

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split

# get data
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/melbourne_house_prices/MELBOURNE_HOUSE_PRICES_LESS.csv')

# drop high cardinality cols
df.drop(columns=['Suburb','Address','SellerG','Date'], inplace=True)

# drop na values in Price
df.dropna(subset=['Price'], inplace=True)

# drop duplicated rows
df.drop_duplicates(inplace=True)



### 1) 데이터의 특성을 파악하기 위해 pandas_profiling을 사용해 보세요.
- **(Urclass Quiz) 범주형 특성으로 원핫인코딩을 할 수 있는 특성의 전체 개수를 제출하세요.**

In [ ]:
df.profile_report()

In [2]:
df.dtypes

Rooms              int64
Type              object
Price            float64
Method            object
Postcode           int64
Regionname        object
Propertycount      int64
Distance         float64
CouncilArea       object
dtype: object

In [ ]:
# Type, Method, Regionname, CoucilArea : 4가지

### 2) 범주형 변수들에 대해 one-hot encoding(category_encoders)을 수행하고 데이터를 train/test 셋으로 무작위로 나누어 주세요.(train/test = 80%/20%)
- [category_encoders.one_hot.OneHotEncoder](https://contrib.scikit-learn.org/category_encoders/onehot.html#one-hot), 파라미터는 use_cat_names = True 사용
- 데이터 분리는 [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train%20test%20split#sklearn-model-selection-train-test-split) 을 사용해 주세요. 사용해야 하는 파라미터는 test_size, train_size, random_state=2 입니다. 나머지는 default 사용합니다.
- **(Urclass Quiz) 분리된 학습데이터의 첫번째 샘플의 인덱스 값를 제출하세요.**


In [6]:
from category_encoders import OneHotEncoder

encoder = OneHotEncoder(use_cat_names = True)
df = encoder.fit_transform(df)

train, test = train_test_split(df, train_size = 0.8, test_size = 0.2, random_state = 2)
train

,Rooms,Type_h,Type_t,Type_u,Price,Method_S,Method_SP,Method_VB,Method_PI,Method_SA,...,CouncilArea_Nillumbik Shire Council,CouncilArea_Whittlesea City Council,CouncilArea_Knox City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra Ranges Shire Council,CouncilArea_Cardinia Shire Council,CouncilArea_Macedon Ranges Shire Council,CouncilArea_Mitchell Shire Council,CouncilArea_Moorabool Shire Council,CouncilArea_Murrindindi Shire Council
602,4,1,0,0,1750000.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40851,5,1,0,0,900000.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57233,4,1,0,0,530000.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8816,4,1,0,0,975000.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3481,3,1,0,0,556000.0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44542,3,1,0,0,695000.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43399,2,0,0,1,490000.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8789,6,1,0,0,830000.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
51700,3,1,0,0,380000.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
target = 'Price'

X_train = train.drop(target, axis = 1)
y_train = train[target]
X_test = test.drop(target, axis = 1)
y_test = test[target]

### 3) SelectKBest를 사용해서 20개의 특성만 selection 하세요
- score_func 에는 f_regression을 사용합니다
- **(Urclass Quiz) 가장 스코어(Attributes: scores_ 사용)가 높게 나온 특성의 이름을 제출하세요.**

In [10]:
from sklearn.feature_selection import f_regression, SelectKBest
import numpy as np

selector = SelectKBest(score_func = f_regression, k = 20)

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

column = df.columns
score = selector.scores_
idx = np.argmax(score)
column[idx]

'Rooms'

### 4) RidgeCV를 사용해서 회귀모델을 만들어 보세요
- attribute는 normalize=True, cv=5를 사용하세요. 나머지는 default
- **(Urclass Quiz) alphas = [0, 0.001, 0.01, 0.1, 1] 을 시도해보고 가장 좋은 점수가 나온 alpha 값을 제출하세요.**

In [ ]:
# 코랩에서 아래와 같은 에러 발생시, sklearn 재설치 진행합니다.
# 에러메시지: 'RidgeCV' object has no attribute 'best_score_' (ver.0.23부터 적용됨. 공식문서에서 확인 가능.)
# 해결: sklearn 재설치 필요. 버전 0.23 이상
# 재설치 방법⇣: 

# sklearn 버전 확인 -> 0.23보다 낮다면 재설치 진행.
import sklearn
sklearn.__version__

# sklearn 재설치
!pip uninstall scikit-learn -y
!pip install -U scikit-learn

# sklearn 버전 재확인 -> 0.23 이상으로 설치되었는지 확인
import sklearn
sklearn.__version__

In [ ]:
from sklearn.linear_model import RidgeCV

alphas = [0,0.001,0.01,0.1,1]

ridge = RidgeCV(alphas = alphas, normalize = True, cv = 5)
ridge.fit(X_train_selected, y_train)

In [26]:
ridge.alpha_, ridge.best_score_

(0.001, 0.5510507608242425)

### 5) 테스트셋에 대한 MAE 값을 계산하고 모델이 잘 만들어졌는지 확인해 보세요.
- **(Urclass Quiz) 테스트셋에 대한 MAE 값을 제출하세요.(소수점 이하 반올림)**

In [28]:
from sklearn.metrics import mean_absolute_error

y_pred = ridge.predict(X_test_selected)

mae = mean_absolute_error(y_test, y_pred)
mae.round(0)

262857.0

## 🔥 도전과제(Github - Discussion)

### 공지로 안내드리는 Github - Discussion 페이지에 올려보세요:)

### 6) 본인만의 아이디어로 best ridge regression model을 만들어 서로 공유하시고 토론해 보세요. 
    - 어떤 특성공학을 사용했고, 그 이유와 기대효과는 무엇이었나요? 
    - 모델 성능 평가 지표는 무엇으로 선정했고, 선정한 이유는 무엇인가요? 
    - 어떠한 머신러닝 라이브러리를 사용했고, 어떤 튜닝을 진행했나요? 테스트셋의 성능 변화는 어떠한가요? 
    - 과제 수행 중 부딪혔던 어려움이나 모르는 점이 있었나요? 있었다면 어떻게 해결해 볼 수 있을까요? 


### 7) 원핫인코딩에서 모든 범주형 값(unique한)을 컬럼으로 인코딩할 경우 어떤 문제점이 발생할 수 있고, 어떻게 해결할 수 있는지에 대하여 조사한 후 함께 논의해 보세요.

### 8) 특성선택(feature selection) 방법들에 대하여 조사한 후 함께 논의해 보세요